# Assignment 2: Bias in data
### Data 512
### Saturday, October 5
### Tara Wilson

Python version?? Imports??

In [1]:
import pandas as pd
import requests
import json
import logging
import numpy as np

In [2]:
logging.basicConfig(level=logging.INFO,
                    datefmt='%m-%d %H:%M',
                    format='%(message)s',
                    filename='bias_in_data_error_log.log',
                    filemode='w')

In [3]:
page_data = pd.read_csv("source_data/page_data.csv")
population_data = pd.read_csv("source_data/WPDS_2018_data.csv")

In [4]:
page_data.head()

,page,country,rev_id
0,Template:ZambiaProvincialMinisters,Zambia,235107991
1,Bir I of Kanem,Chad,355319463
2,Template:Zimbabwe-politician-stub,Zimbabwe,391862046
3,Template:Uganda-politician-stub,Uganda,391862070
4,Template:Namibia-politician-stub,Namibia,391862409


In [5]:
population_data.head()

,Geography,Population mid-2018 (millions)
0,AFRICA,"1,284"
1,Algeria,42.7
2,Egypt,97
3,Libya,6.5
4,Morocco,35.2


Rows with page names that begin with the string "Template" need to be filtered out of `page_data` as these are not Wikipedia articles and we do not want to include them in the anlysis.

In [44]:
page_data = page_data[~page_data["page"].str.startswith("Template")]
page_data.head()

,page,country,rev_id
1,Bir I of Kanem,Chad,355319463
10,Information Minister of the Palestinian Nation...,Palestinian Territory,393276188
12,Yos Por,Cambodia,393822005
23,Julius Gregr,Czech Republic,395521877
24,Edvard Gregr,Czech Republic,395526568


## ****** NOTE AOUT CAPITAL POPULATION!!!

## ORES Requests

In [7]:
def get_ores_data(revision_ids):
    headers = {'User-Agent' : 'https://github.com/TaraWilson17', 'From' : 'wwtara@uw.edu'}
    
    endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/?models={model}&revids={revids}'
    
    params = {'project' : 'enwiki',
              'model'   : 'wp10',
              'revids'  : '|'.join(str(x) for x in revision_ids)
              }
    api_call = requests.get(endpoint.format(**params))
    response = api_call.json()
    return response

In [8]:
page_data.shape[0]

46701

In [12]:
revision_id = []
article_quality = []

page_data["rev_id"] = page_data["rev_id"].astype(np.int64)

for i in range(0, page_data.shape[0], 50):
    ores_responses = get_ores_data(np.array(page_data["rev_id"].iloc[i:i + 50,]))
    for article in ores_responses["enwiki"]["scores"]:
        try:
            article_quality.append(ores_responses["enwiki"]["scores"][article]["wp10"]["score"]["prediction"])
        except:
            logging.info("Unable to get a ORES response for revision id: %s", article)
        else:
            revision_id.append(article)

516633096
550682925
627547024
671484594
684023803
684023859
698572327
699260156
703773782
706810694
709508670
711288191
712872338
712872421
712872473
712872531
712873183
712873308
712873386
712878000
712878267
712878343
712878396
712881543
712881676
712881741
712881882
712889562
712889594
712889683
712889781
712889809
712891291
712891354
712891378
712891476
713368646
714352602
715273866
715534283
717927381
719521006
719581803
720054719
720356159
720688837
721509220
724052271
726600165
730950147
734957625
736660951
737123608
738514517
738984692
747688056
747922835
749326717
754739349
756697478
757313957
757961591
760194556
760195376
763558111
764019224
764911200
765662083
766101638
768013050
768871687
771213598
771642775
773355081
774023957
774293013
777163201
779101752
779135011
779954797
779957437
782170063
783382630
786721553
787181453
787391918
787398581
788310383
788722110
788844423
789080627
789271422
789281061
789285762
789286413
790028876
790147995
791866288
792400552
792857662


In [22]:
article_data = pd.DataFrame()
article_data["revision_id"] = revision_id
article_data["article_quality"] = article_quality
article_data.head()

,revision_id,article_quality
0,355319463,Stub
1,393276188,Stub
2,393822005,Stub
3,395521877,Stub
4,395526568,Stub


In [27]:
article_data["revision_id"] = article_data["revision_id"].astype(str).astype(int)
all_article_data = pd.merge(article_data, page_data, left_on="revision_id", right_on="rev_id")
all_article_data = all_article_data.drop(columns=["rev_id"])
all_article_data.head()

,revision_id,article_quality,page,country
0,355319463,Stub,Bir I of Kanem,Chad
1,393276188,Stub,Information Minister of the Palestinian Nation...,Palestinian Territory
2,393822005,Stub,Yos Por,Cambodia
3,395521877,Stub,Julius Gregr,Czech Republic
4,395526568,Stub,Edvard Gregr,Czech Republic


In [34]:
all_data = pd.merge(all_article_data, population_data, left_on="country", right_on="Geography")
all_data = all_data.drop(columns=["Geography"])
all_data = all_data.rename(columns={"Population mid-2018 (millions)": "population"})
all_data.head()

,revision_id,article_quality,page,country,population
0,355319463,Stub,Bir I of Kanem,Chad,15.4
1,498683267,Stub,Abdullah II of Kanem,Chad,15.4
2,565745353,Stub,Salmama II of Kanem,Chad,15.4
3,565745365,Stub,Kuri I of Kanem,Chad,15.4
4,565745375,Stub,Mohammed I of Kanem,Chad,15.4


In [35]:
all_data.to_csv("wp_wpds_politicians_by_country.csv", sep=",", columns=["country", "article_name", "revision_id", "article_quality", "population"])

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


## Analysis

## Result tables

### 1. Top 10 countries by coverage

### 2. Bottom 10 countries by coverage

### 3. Top 10 countries by relative quality

### 4. Bottom 10 countries by relative quality

### 5. Geographic regions by coverage (by politician articles from countries in each region as a proportion of total regional population)

### 6. Geographic regions by coverage (by relative proportion of politician articles from countries in each region that are of GA and FA-quality)

## Reflections and implications